In [1]:
import json
import requests

# Ryan code starts

In [10]:
url = "https://api.covid19api.com/all"
response = requests.get(url)
with open("covid_all.json", 'w') as jsonfile:
    json.dump(response.json(), jsonfile)

In [12]:
with open("static/data/covid_all.json") as f:
    covid = json.load(f)

AttributeError: 'list' object has no attribute 'head'

In [14]:
#code to store data in MongoDB
import pymongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.covid_db
for i in range(len(covid)):
    db.historical.insert_one(covid[i])

In [15]:
confirmed = []
deaths = []
recovered = []

In [16]:
for i in range(len(covid)):
    data = covid[i]
    if data['Status'] == 'confirmed':
        confirmed.append(data)
    elif data['Status'] == 'deaths':
        deaths.append(data)
    elif data['Status'] == 'recovered':
        recovered.append(data)

KeyError: 'Status'

In [18]:
def get_date_list(list):
    dates = []
    for i in range(len(list)):
        if list[i]['Date'] not in dates:
            dates.append(list[i]['Date'])
    return sorted(dates)

In [19]:
dates_confirmed = []
dates_deaths = []
dates_recovered = []
dates_confirmed = get_date_list(confirmed)
dates_deaths = get_date_list(deaths)
dates_recovered = get_date_list(recovered)

In [20]:
covid_clean = {
    'confirmed': {},
    'deaths': {},
    'recovered': {}    
}

In [21]:
def get_date_data(dates, data_list, status):
    for i in range(len(dates)):
        current = []
        for j in range(len(data_list)):
            if dates[i] == data_list[j]["Date"]:
                current.append(data_list[j])
        covid_clean[status][dates[i]] = current
    return covid_clean

In [22]:
covid_clean = get_date_data(dates_confirmed, confirmed, 'confirmed')
covid_clean = get_date_data(dates_deaths, deaths, 'deaths')
covid_clean = get_date_data(dates_recovered, recovered, 'recovered')

In [ ]:
# covid_clean
# for i in range(len(dates_deaths)):
#     print(len(covid_clean['deaths'][dates_deaths[i]]))

In [ ]:
# import pymongo
# conn = 'mongodb://localhost:27017'
# client = pymongo.MongoClient(conn)
# db = client.covid_db
# db.covid_clean.insert_one(covid_clean)  ## this is not working due to oversize

# db.covid_confirmed.insert_one(covid_clean['confirmed'])
# db.covid_deaths.insert_one(covid_clean['deaths'])
# db.covid_recovered.insert_one(covid_clean['recovered'])

# Ryan code ends

# Kana code starts

In [3]:
import pandas as pd
from datetime import datetime
import numpy as np

In [4]:
# Read unemployment claims data
claim_data = pd.read_csv("resources/unemployment_by_state.csv", parse_dates=True)
claim_data.head()

,State,Filed week ended,Initial Claims,Reflecting Week Ended,Continued Claims,Covered Employment,Insured Unemployment Rate
0,Alabama,1/5/2019,"6,660",12/29/2018,"22,085","1,894,608",1.17
1,Alabama,1/12/2019,"4,386",1/5/2019,"20,075","1,894,608",1.06
2,Alabama,1/19/2019,"3,217",1/12/2019,"17,574","1,894,608",0.93
3,Alabama,1/26/2019,"2,414",1/19/2019,"16,904","1,894,608",0.89
4,Alabama,2/2/2019,"2,505",1/26/2019,"16,343","1,894,608",0.86


In [5]:
# Select needed columns
claim_data = claim_data[['State','Filed week ended','Initial Claims']]
claim_data.head()

,State,Filed week ended,Initial Claims
0,Alabama,1/5/2019,"6,660"
1,Alabama,1/12/2019,"4,386"
2,Alabama,1/19/2019,"3,217"
3,Alabama,1/26/2019,"2,414"
4,Alabama,2/2/2019,"2,505"


In [6]:
# Replace comma with empty space
claim_data = claim_data.stack().str.replace(',','').unstack()
claim_data.head()

,State,Filed week ended,Initial Claims
0,Alabama,1/5/2019,6660
1,Alabama,1/12/2019,4386
2,Alabama,1/19/2019,3217
3,Alabama,1/26/2019,2414
4,Alabama,2/2/2019,2505


In [7]:
# Change the date format to be able to sort
claim_data['Filed week ended'] = pd.to_datetime(claim_data['Filed week ended'])

# Retrive data only after 1/1/2020
filtered_claim_data = claim_data.loc[claim_data["Filed week ended"] > "2020-01-01"]

# Remove timestamp
filtered_claim_data["Filed week ended"] = filtered_claim_data["Filed week ended"].map(lambda ts: ts.strftime("%Y-%m-%d"))

# Drop na
filtered_claim_data = filtered_claim_data.dropna()
filtered_claim_data.head()

/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,State,Filed week ended,Initial Claims
52,Alabama,2020-01-04,4578
53,Alabama,2020-01-11,3629
54,Alabama,2020-01-18,2483
55,Alabama,2020-01-25,2129
56,Alabama,2020-02-01,2170


In [8]:
# Change data type to int
filtered_claim_data['Initial Claims'] = filtered_claim_data['Initial Claims'].astype(int)

# Check data type again
filtered_claim_data.dtypes

State               object
Filed week ended    object
Initial Claims       int64
dtype: object

In [9]:
# Create a dictionary for claims by state for each date
df = filtered_claim_data.groupby(["Filed week ended", "State"]).first().reset_index()
datelist = filtered_claim_data["Filed week ended"].unique()
claims_jsonfile = {}

for i in range(len(datelist)):
    claims_jsonfile[datelist[i]] = list()
    for j in range(len(df)):
        if datelist[i] == df["Filed week ended"].iloc[j]:
            state = df['State'][j]
            number = df['Initial Claims'][j]
            claims_jsonfile[datelist[i]].append({state:number}) 
claims_jsonfile

{'2020-01-04': [{'Alabama': 4578},
  {'Alaska': 981},
  {'Arizona': 3132},
  {'Arkansas': 2576},
  {'California': 36720},
  {'Colorado': 2431},
  {'Connecticut': 7852},
  {'Delaware': 875},
  {'District of Columbia': 481},
  {'Florida': 4618},
  {'Georgia': 17060},
  {'Hawaii': 1825},
  {'Idaho': 2957},
  {'Illinois': 11779},
  {'Indiana': 4434},
  {'Iowa': 4586},
  {'Kansas': 2019},
  {'Kentucky': 5175},
  {'Louisiana': 1658},
  {'Maine': 1663},
  {'Maryland': 3997},
  {'Massachusetts': 9464},
  {'Michigan': 13207},
  {'Minnesota': 7293},
  {'Mississippi': 1178},
  {'Missouri': 7474},
  {'Montana': 1555},
  {'Nebraska': 1099},
  {'Nevada': 2264},
  {'New Hampshire': 690},
  {'New Jersey': 15325},
  {'New Mexico': 799},
  {'New York': 44846},
  {'North Carolina': 3508},
  {'North Dakota': 836},
  {'Ohio': 10759},
  {'Oklahoma': 2943},
  {'Oregon': 6714},
  {'Pennsylvania': 27298},
  {'Puerto Rico': 853},
  {'Rhode Island': 1972},
  {'South Carolina': 7471},
  {'South Dakota': 398},
  {

In [ ]:
## Kana to solve mystery later##
# for key, value in states_clean.items():
#     current = []
#     for j in range(len(filtered_claim_data['State'])):
#         if filtered_claim_data['State'].iloc[j] == key:
#             state_claim_dict = {}
#             state_claim_dict[filtered_claim_data['Filed week ended'][j]] =filtered_claim_data['Initial Claims'][j]
#             current.append(state_claim_dict)
#             states_clean[key] = current
# print(states_clean)

# for key, value in dates_clean.items():
#     current = []
#     for j in range(len(filtered_claim_data['Filed week ended'])):
#         if filtered_claim_data['Filed week ended'].iloc[j] == key:
#             state_claim_dict = {}
#             print(key)
# #             print(filtered_claim_data['Filed week ended'].iloc[j] )
        
# #             print(state_claim_dict[filtered_claim_data['State'].iloc[j]])
#             print(filtered_claim_data['Initial Claims'][j])
#             state_claim_dict[filtered_claim_data['State'].iloc[j]] =filtered_claim_data['Initial Claims'][j]
#             current.append(state_claim_dict)
#             dates_clean[key] = current
# print(dates_clean)

# Kana code ends

# Connor code starts

# Connor code ends

# Nabeel code starts

# Nabeel code ends